<a href="https://colab.research.google.com/github/Bjoyita/ThermoelectricDataAnalysis/blob/main/SHAP_PF_kappa_Github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing Libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns

#Installing Materials Libraries
pip install pymatgen
pip install matminer
pip install shap

#Loading Data
ds_RT = pd.read_csv('Pf_kappa_RT_1.csv')
ds_RT

from pymatgen.core.composition import Composition
from matminer.featurizers.composition import ElementFraction
ef = ElementFraction()
element_fraction_labels = ef.feature_labels()
print(element_fraction_labels)
#Preprocessing of Data
ds_RT['Formula']
Comp = []
for value in ds_RT['Formula']:
  Comp.append(Composition(value))
Comp

ds_RT['Composition'] = Comp
ds_RT

ds_RT = ef.featurize_dataframe(ds_RT,'Composition')
ds_RT

ds_RT = ds_RT.loc[:, (ds_RT != 0).any(axis=0)]
ds_RT

ds_RT = ds_RT.drop(['Composition'], axis = 1)
ds_RT

ds_RT.columns

ds_RT = ds_RT.reindex(columns = ['Formula','B', 'O', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'K', 'Ca',
       'Ti', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'Se', 'Sr',
       'Y', 'Zr', 'Nb', 'Mo', 'Ag', 'In', 'Sn', 'Sb', 'Te', 'I', 'Cs', 'Ba',
       'La', 'Ce', 'Nd', 'Sm', 'Gd', 'Tb', 'Dy', 'Ho', 'Yb', 'Hf', 'W', 'Au',
       'Tl', 'Pb', 'Bi', 'kappa', 'PF'])

ds_RT
#Model fitting: Random Forest algorithm
#Correlation of elements with PF
X = ds_RT.iloc[:, 1: 50]
y_pf = ds_RT.iloc[:, -1]

from sklearn.model_selection import train_test_split
X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(X, y_pf, test_size = 0.2, random_state = 0)
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 50, random_state = 0)
rf.fit(X_train_p, y_train_p)

# SHAP Values for PF
import shap
explainer = shap.TreeExplainer(rf)
shap_values_pf = explainer.shap_values(X_train_p)
print(shap_values_pf)
shap.summary_plot(shap_values_pf, X_train_p, plot_type="bar") #This gives the absolute SHAP values
shap.summary_plot(shap_values_pf, X_train_p)